# S3 Transfer Acceleration

S3 transfer acceleration is enabled at the bucket level, then leveraged by client via a 
special url indicating the use of xfer acceleration.

In [ ]:
# SDK set up
import boto3
import random

s3client = boto3.client('s3')
my_session = boto3.session.Session()
my_region = my_session.region_name
print 'working in {}'.format(my_region)

In [ ]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-e1-' + str(random.randint(1, 9999999))
print bucket_name

In [ ]:
# Create the bucket
response = s3client.create_bucket(
    Bucket=bucket_name
)

print response

In [ ]:
# Create a file to upload to the bucket
f = open("./hello.txt","w")
f.write("hello, world")
f.close()

In [ ]:
# What happens if we try to upload to the special endpoint before enabling 
# transfer acceleration?

# You will get an error that looks like:
# upload failed: ./hello.txt to s3://ds-e1-8927099/sample An error occurred 
# (InvalidRequest) when calling the PutObject operation: S3 Transfer Acceleration 
# is not configured on this bucket

In [ ]:
%%bash -s "$bucket_name" "$my_region"
aws s3 cp hello.txt s3://$1/sample --region $2 --endpoint-url https://s3-accelerate.amazonaws.com

In [ ]:
# Enable transfer acceleration on the bucket
response = s3client.put_bucket_accelerate_configuration(
    Bucket=bucket_name,
    AccelerateConfiguration={
        'Status': 'Enabled'
    }
)

print response

In [ ]:
# Now we can upload using the special URL. Note according to the documentation the url starts with bucket name,
# but the below URL works with the cli

In [ ]:
%%bash -s "$bucket_name" "$my_region"
aws s3 cp hello.txt s3://$1/sample --region $2 --endpoint-url https://s3-accelerate.amazonaws.com

In [ ]:
# Empty the bucket
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='sample'
)

print response

In [ ]:
# Delete the bucket
response = s3client.delete_bucket(
    Bucket=bucket_name
)

print response